# 1. Imports and settings

In [2]:
# Import necessary libraries
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from google.colab import drive

# Mount Google Drive to access DIV2K dataset
drive.mount('/content/drive')

# Paths to dataset
DIV2K_DIR = "/content/drive/My Drive/DIV2K"
DIV2K_train_HR = os.path.join(DIV2K_DIR, "DIV2K_train_HR")
DIV2K_valid_HR = os.path.join(DIV2K_DIR, "DIV2K_valid_HR")

# Check if dataset folders exist
if not os.path.exists(DIV2K_train_HR):
    raise FileNotFoundError(f"Training folder not found: {DIV2K_train_HR}")
if not os.path.exists(DIV2K_valid_HR):
    print(f"Validation folder not found: {DIV2K_valid_HR}, proceeding with training data split.")
print("Paths are set correctly!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Paths are set correctly!


# 2. Loading and preparing the data
Download and process DIV2K dataset

In [3]:
# Function to load and preprocess images
def load_images(path, target_size=(256, 256)):
    images = []
    for filename in os.listdir(path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img = load_img(os.path.join(path, filename), target_size=target_size)
            img = img_to_array(img) / 255.0  # Normalize images to [0, 1]
            images.append(img)
    return np.array(images)

# Function to downscale images
def downscale_images(images, scale=2):
    LR_images = []
    for img in images:
        h, w, _ = img.shape
        LR = cv2.resize(img, (w // scale, h // scale), interpolation=cv2.INTER_AREA)
        LR_images.append(LR)
    return np.array(LR_images)

# Load HR training images
HR_images_train = load_images(DIV2K_train_HR)
LR_images_train = downscale_images(HR_images_train)

# Load HR validation images if available, otherwise split training data
if os.path.exists(DIV2K_valid_HR):
    HR_images_valid = load_images(DIV2K_valid_HR)
    LR_images_valid = downscale_images(HR_images_valid)
else:
    HR_images_train, HR_images_valid = train_test_split(HR_images_train, test_size=0.2, random_state=42)
    LR_images_train, LR_images_valid = train_test_split(LR_images_train, test_size=0.2, random_state=42)

# Apply data augmentation to training data
datagen = ImageDataGenerator(rotation_range=15, horizontal_flip=True, vertical_flip=True)
batch_size = 32
HR_augmented = datagen.flow(HR_images_train, batch_size=batch_size, shuffle=True)
LR_augmented = datagen.flow(LR_images_train, batch_size=batch_size, shuffle=True)


# 3. Building the Super-Resolution model

In [4]:
# Load VGG16 model once (outside the perceptual_loss function)
vgg = VGG16(include_top=False, weights='imagenet', input_shape=(None, None, 3))
vgg.trainable = False  # Freeze VGG16 layers
feature_extractor = tf.keras.Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)

# Define perceptual loss function
def perceptual_loss(y_true, y_pred):
    true_features = feature_extractor(y_true)
    pred_features = feature_extractor(y_pred)
    return tf.reduce_mean(tf.square(true_features - pred_features))

# Function to create an improved Super-Resolution model
def build_improved_model():
    inputs = Input(shape=(None, None, 3))
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    for _ in range(5):  # Add 5 residual blocks
        residual = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        residual = Conv2D(64, (3, 3), activation='relu', padding='same')(residual)
        x = Add()([x, residual])  # Add residual connection
    x = UpSampling2D(size=(2, 2))(x)  # Upsample the output
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    outputs = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss=perceptual_loss)
    return model

# Build the model
model = build_improved_model()
model.summary()  # Display the model architecture



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, None, None, 3)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, None, None, 64) │          1,792 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, None, None, 64) │         36,928 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, None, None, 64) │         36,928 │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, None, None, 64) │              0 │ conv2d[0][0],          │
│                           │                        │                │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, None, None, 64) │         36,928 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, None, None, 64) │         36,928 │ conv2d_3[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, None, None, 64) │              0 │ add[0][0],             │
│                           │                        │                │ conv2d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, None, None, 64) │         36,928 │ add_1[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, None, None, 64) │         36,928 │ conv2d_5[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, None, None, 64) │              0 │ add_1[0][0],           │
│                           │                        │                │ conv2d_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, None, None, 64) │         36,928 │ add_2[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, None, None, 64) │         36,928 │ conv2d_7[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_3 (Add)               │ (None, None, None, 64) │              0 │ add_2[0][0],           │
│                           │                        │                │ conv2d_8[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_9 (Conv2D)         │ (None, None, None, 64) │         36,928 │ add_3[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_10 (Conv2D)        │ (None, None, None, 64) │         36,928 │ conv2d_9[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_4 (Add)               │ (None, None, None, 64) │              0 │ add_3[0][0],           │
│                      

 Total params: 409,731 (1.56 MB)

 Trainable params: 409,731 (1.56 MB)

 Non-trainable params: 0 (0.00 B)

# 4. Training the model

In [ ]:
# Split data into training and validation sets
LR_train, LR_val, HR_train, HR_val = train_test_split(LR_images_train, HR_images_train, test_size=0.2, random_state=42)

# Define callbacks for training
callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True),
    ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss')
]

# Train the model
history = model.fit(
    LR_train, HR_train,
    validation_data=(LR_val, HR_val),
    batch_size=batch_size,
    epochs=20,
    callbacks=callbacks,
    shuffle=True
)

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Training Progress')
plt.show()


Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 2433s 122s/step - loss: 43.4683 - val_loss: 27.2443
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 2432s 119s/step - loss: 24.2393 - val_loss: 17.5549
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 2420s 119s/step - loss: 15.6907 - val_loss: 12.5286
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 2411s 121s/step - loss: 11.4580 - val_loss: 9.5349
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 2382s 120s/step - loss: 9.1792 - val_loss: 8.3831
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 2418s 120s/step - loss: 8.0280 - val_loss: 7.8214
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 2432s 120s/step - loss: 7.4747 - val_loss: 7.4113
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 2413s 120s/step - loss: 7.1675 - val_loss: 7.0539
Epoch 9/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 2429s 120s/step - loss: 6.9109 - val_loss: 6.8219
Epoch 10/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 2423s 120s/step - loss: 6.8179 - val_loss: 6.6304
Epoch 11/20
18/20 ━━━━━━━━━━━━━━━━━━━━ 3:33 107s/step - loss: 6.4747

# 5. Testing the model
Compare results with cv2.resize

In [ ]:
# Function to upscale an image using the trained model
def upscale_image(model, LR_image):
    LR_image = np.expand_dims(LR_image, axis=0)  # Add batch dimension
    SR_image = model.predict(LR_image)[0]  # Predict and remove batch dimension
    SR_image = np.clip(SR_image * 255.0, 0, 255).astype('uint8')  # Denormalize
    return SR_image

# Function to display results
def display_results(LR, SR, HR, Interpolated):
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    axes[0].imshow(LR)
    axes[0].set_title("Low Resolution")
    axes[1].imshow(Interpolated)
    axes[1].set_title("Interpolation (Cubic)")
    axes[2].imshow(SR)
    axes[2].set_title("Super Resolution")
    axes[3].imshow(HR)
    axes[3].set_title("High Resolution")
    for ax in axes:
        ax.axis('off')
    plt.tight_layout()
    plt.show()

# Test on a random validation sample
index = np.random.randint(len(LR_val))
LR_sample = LR_val[index]
HR_sample = HR_val[index]

# Super-Resolution using the trained model
SR_sample = upscale_image(model, LR_sample)

# Traditional interpolation (cv2.resize)
interpolated_sample = cv2.resize(
    LR_sample, (HR_sample.shape[1], HR_sample.shape[0]), interpolation=cv2.INTER_CUBIC
)

# Display the results
display_results(LR_sample, SR_sample, HR_sample, interpolated_sample)


In [ ]:
# Save the trained model for future use
model.save("super_resolution_model.h5")